In [16]:
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('..')

In [17]:
from omegaconf import OmegaConf
import torch
import segmentation_models_pytorch as smp
from torch import nn
from pytorch_lightning.logging import TensorBoardLogger
from dl.lightning_model import *

In [18]:
model = smp.FPN(encoder_name="resnext50_32x4d", classes=2, decoder_merge_policy='cat')
# class_weight = dataset["train"].numpixels[1,0:2]
# class_weight = torch.from_numpy(1-class_weight/class_weight.sum()).type('torch.FloatTensor')

criterion = nn.CrossEntropyLoss(
    ignore_index = -100,
    reduction = 'none'
)

In [19]:
conf = OmegaConf.create(
    {        
        'dataloader_args': {
            'batch_size':6,
            'n_classes':2
        },
        
        'optimizer_args':{
            'lr':0.001,
            'encoder_lr':0.0005,
            'weight_decay':0.0003,
            'encoder_weight_decay':0.00003
        },
        
        'scheduler_args': {
            'factor':0.25,
            'patience':2
        },
    
        'loss_args' : {
            'edge_weight' : 1.1,
        }

    }
)
print(conf.pretty())

dataloader_args:
  batch_size: 6
  n_classes: 2
optimizer_args:
  lr: 0.001
  encoder_lr: 0.0005
  weight_decay: 0.0003
  encoder_weight_decay: 3.0e-05
scheduler_args:
  factor: 0.25
  patience: 2
loss_args:
  edge_weight: 1.1



In [20]:
logdir = "../../../../results/tensorboard_logs/"
model_dir = "../../../../results/trained_models/"

In [21]:
%load_ext tensorboard
%tensorboard --logdir {logdir}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 17604), started 2:32:57 ago. (Use '!kill 17604' to kill it.)

In [22]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filepath = model_dir,
    save_top_k = 1,
    save_last = True,
    verbose = True, 
    monitor = 'val_loss',
    mode = 'min',
    prefix = ''
)

logger = TensorBoardLogger(save_dir=logdir, name="FPN")

In [23]:
lightning_model = SegModel(model=model, criterion = criterion, hparams = conf, dataset='kumar')
trainer = pl.Trainer(
    default_root_dir=model_dir,
    max_epochs=30, 
    gpus=0, 
    checkpoint_callback=checkpoint_callback, 
    logger=logger
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [24]:
#trainer.logger.log_dir

In [25]:
#trainer.fit(lightning_model)